Google Play and App Store analysis of ad revenue

This notebook attempts to identify apps that are likely to attract more users.

In [1]:
from csv import reader

### The Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
print(android_header)
print('\n')
explore_data(android, 0, 3)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']




In [4]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']




OK, so the headings are somewhat helpful. No sense leaving anything up for discussion though.  Let's link to the actual data descriptions.

iOS data description: https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps
Android data description: https://www.kaggle.com/lava18/google-play-store-apps



DATA CLEANING

In [5]:
##Do all of the rows have the same number of entries?

android_header_len = len(android_header)

for row in android:
    if len(row) < android_header_len:
        print(row) #data from a problem row
        print(android.index(row)) #the row index

        
print('\n')

ios_header_len = len(ios_header)

for row in ios:
    if len(row) < ios_header_len:
        print(row) #data from a problem row
        print(ios.index(row)) #the row index

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10472




That's a problem. Let's get rid of that row for the moment.  Alternatively, we could dig into the missing field and either find the appropriate value(s) or impute based on the rest of the data. An exercise for another day.

In [6]:
del android[10472]

Now that our data is structurally sound, we should look for things like duplication. We will start with a quick scan over the app names to see if anything jumps out.

In [7]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Duplicate apps: ', len(duplicate_apps))
print('\n')
print('Examples: ', duplicate_apps[:15])

Duplicate apps:  1181


Examples:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [8]:
for app in android:
    name = app[0]
    if name == 'Slack':
        print(app)

['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


In [9]:
android_header

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

Nearly everything is identical, however the 4th column (# of Reviews) appears to be something we can use to define a distinct entry. For all of the duplicate entries, let's keep the one with the most reviews (which should presumably be the most recent data) and get rid of the rest.

In [10]:
android_reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in android_reviews_max and android_reviews_max[name] < n_reviews:
        android_reviews_max[name] = n_reviews
    
    if name not in android_reviews_max:
        android_reviews_max[name] = n_reviews
        
#len(android_reviews_max)

The above code could have been written more eloquently (we're doing the same thing in each IF statement). If we find an app entry that has a higher # of reviews, we're replacing the # of reviews in our dictionary. If there's no entry, we're adding one.  This is going to help us whittle down the main dataset shortly.

Below, we're looping through all of the rows in the dataset. If the entry matches the max # of reviews from the dictionary we just built, we keep it in a cleaned dataset. If not, we essentially discard it.

In [11]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])

    
    if (n_reviews == android_reviews_max[name]) and name not in already_added:
            android_clean.append(app)
            already_added.append(name)

print(len(android_clean))
print(len(already_added))

9659
9659


This is one place where knowing SQL would be really handy. If the data were in a  SQL database, we could easily grab it like this:

SELECT *, ROW_NUMBER() OVER (PARTITION BY name ORDER BY ratings DESC)
FROM android_data
HAVING ROW_NUMBER() = 1


We're only interested in looking at English apps. Normally, you could filter on the ASCII code being <127 (a-z, A-Z, 0-9, and some special characters) although with the proliferation of emojis, we're going to need to provide some leeway. Let's set a cutoff at 3 non-standard characters.  Any more than that and we will exclude the app.

In [12]:
def char_check(app_name):
    
    non_english = 0
    
    for character in app_name:
        if ord(character) > 127:
            non_english += 1

    if non_english > 3:
        return False
    else:
        return True

In [13]:
char_check('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [14]:
fully_cleaned_data = []    

for app in android_clean:
    name = app[0]

    
    if (char_check(name) == True):
        fully_cleaned_data.append(app)

In [15]:
len(fully_cleaned_data)

9614

In [16]:
fully_cleaned_data_free = []

for app in fully_cleaned_data:
    if (app[7] == '0'):
        fully_cleaned_data_free.append(app)

In [17]:
len(fully_cleaned_data_free)

8864

Now do the same thing for ios

In [18]:
ios_duplicate_apps = []
ios_unique_apps = []

for app in ios:
    name = app[0]
    if name in ios_unique_apps:
        ios_duplicate_apps.append(name)
    else:
        ios_unique_apps.append(name)

print('Duplicate apps: ', len(ios_duplicate_apps))
print('\n')
print('Examples: ', ios_duplicate_apps[:15])

Duplicate apps:  0


Examples:  []


In [19]:
char_check('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [20]:
ios_fully_cleaned_data = []    

for app in ios:
    name = app[0]

    
    if (char_check(name) == True):
        ios_fully_cleaned_data.append(app)

In [21]:
len(ios_fully_cleaned_data)

7197

In [22]:
ios_fully_cleaned_data_free = []

for app in ios_fully_cleaned_data:
    if (app[4] == '0.0'):
        ios_fully_cleaned_data_free.append(app)

In [23]:
len(ios_fully_cleaned_data_free)

4056

In [24]:
print(len(fully_cleaned_data_free))
print(len(ios_fully_cleaned_data_free))


8864
4056


Let's build a couple helper functions to build frequency tables.

In [25]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [26]:
def freq_table(dataset, index):
    freq = {}
    total = len(dataset)

    for row in dataset:
        index_name = row[index]

        if index_name not in freq:
            freq[index_name] = 1
        else:
            freq[index_name] += 1

    for row in freq:
        freq[row] /= total

    return freq

In [27]:
display_table(fully_cleaned_data_free, 1) #Android - Category

FAMILY : 0.18907942238267147
GAME : 0.09724729241877256
TOOLS : 0.08461191335740072
BUSINESS : 0.04591606498194946
LIFESTYLE : 0.039034296028880866
PRODUCTIVITY : 0.03892148014440433
FINANCE : 0.03700361010830325
MEDICAL : 0.03531137184115524
SPORTS : 0.03395758122743682
PERSONALIZATION : 0.03316787003610108
COMMUNICATION : 0.032378158844765345
HEALTH_AND_FITNESS : 0.030798736462093863
PHOTOGRAPHY : 0.02944494584837545
NEWS_AND_MAGAZINES : 0.027978339350180504
SOCIAL : 0.026624548736462094
TRAVEL_AND_LOCAL : 0.023352888086642598
SHOPPING : 0.022450361010830325
BOOKS_AND_REFERENCE : 0.021435018050541516
DATING : 0.01861462093862816
VIDEO_PLAYERS : 0.017937725631768955
MAPS_AND_NAVIGATION : 0.013989169675090252
FOOD_AND_DRINK : 0.012409747292418772
EDUCATION : 0.011620036101083033
ENTERTAINMENT : 0.009589350180505414
LIBRARIES_AND_DEMO : 0.009363718411552346
AUTO_AND_VEHICLES : 0.009250902527075812
HOUSE_AND_HOME : 0.008235559566787004
WEATHER : 0.008009927797833934
EVENTS : 0.0071074007

In [28]:
display_table(fully_cleaned_data_free, 9) #Android - Genre

Tools : 0.08449909747292418
Entertainment : 0.06069494584837545
Education : 0.05347472924187725
Business : 0.04591606498194946
Productivity : 0.03892148014440433
Lifestyle : 0.03892148014440433
Finance : 0.03700361010830325
Medical : 0.03531137184115524
Sports : 0.03463447653429603
Personalization : 0.03316787003610108
Communication : 0.032378158844765345
Action : 0.03102436823104693
Health & Fitness : 0.030798736462093863
Photography : 0.02944494584837545
News & Magazines : 0.027978339350180504
Social : 0.026624548736462094
Travel & Local : 0.023240072202166066
Shopping : 0.022450361010830325
Books & Reference : 0.021435018050541516
Simulation : 0.020419675090252706
Dating : 0.01861462093862816
Arcade : 0.018501805054151624
Video Players & Editors : 0.017712093862815883
Casual : 0.01759927797833935
Maps & Navigation : 0.013989169675090252
Food & Drink : 0.012409747292418772
Puzzle : 0.01128158844765343
Racing : 0.009927797833935019
Role Playing : 0.009363718411552346
Libraries & Demo 

In [29]:
ios_header

['id',
 'track_name',
 'size_bytes',
 'currency',
 'price',
 'rating_count_tot',
 'rating_count_ver',
 'user_rating',
 'user_rating_ver',
 'ver',
 'cont_rating',
 'prime_genre',
 'sup_devices.num',
 'ipadSc_urls.num',
 'lang.num',
 'vpp_lic']

In [30]:
display_table(ios_fully_cleaned_data_free, 11) #ios - prime_genre

Games : 0.5564595660749507
Entertainment : 0.08234714003944774
Photo & Video : 0.04117357001972387
Social Networking : 0.035256410256410256
Education : 0.03254437869822485
Shopping : 0.029832347140039447
Utilities : 0.02687376725838264
Lifestyle : 0.023175542406311637
Finance : 0.020710059171597635
Sports : 0.01947731755424063
Health & Fitness : 0.01873767258382643
Music : 0.016518737672583828
Book : 0.016272189349112426
Productivity : 0.015285996055226824
News : 0.014299802761341223
Travel : 0.013806706114398421
Food & Drink : 0.010601577909270217
Weather : 0.007642998027613412
Reference : 0.004930966469428008
Navigation : 0.004930966469428008
Business : 0.004930966469428008
Catalogs : 0.0022189349112426036
Medical : 0.0019723865877712033


In [31]:
ios_freq_table = freq_table(ios_fully_cleaned_data_free, 11)
print(ios_freq_table)

{'Business': 0.004930966469428008, 'Education': 0.03254437869822485, 'Travel': 0.013806706114398421, 'Catalogs': 0.0022189349112426036, 'Medical': 0.0019723865877712033, 'News': 0.014299802761341223, 'Shopping': 0.029832347140039447, 'Weather': 0.007642998027613412, 'Navigation': 0.004930966469428008, 'Finance': 0.020710059171597635, 'Photo & Video': 0.04117357001972387, 'Music': 0.016518737672583828, 'Food & Drink': 0.010601577909270217, 'Book': 0.016272189349112426, 'Lifestyle': 0.023175542406311637, 'Entertainment': 0.08234714003944774, 'Social Networking': 0.035256410256410256, 'Reference': 0.004930966469428008, 'Health & Fitness': 0.01873767258382643, 'Sports': 0.01947731755424063, 'Games': 0.5564595660749507, 'Productivity': 0.015285996055226824, 'Utilities': 0.02687376725838264}


In [32]:
for genre in ios_freq_table:
    total_rating_points = 0
    len_genre = 0
    
    for row in ios_fully_cleaned_data_free:
        genre_app = row[11]
        
        if genre_app == genre:
            rating = float(row[5])
            total_rating_points += rating
            len_genre += 1

    print(genre +': ' + str(total_rating_points/len_genre))

Business: 6367.8
Education: 6266.333333333333
Travel: 20216.01785714286
Catalogs: 1779.5555555555557
Medical: 459.75
News: 15892.724137931034
Shopping: 18746.677685950413
Weather: 47220.93548387097
Navigation: 25972.05
Finance: 13522.261904761905
Photo & Video: 27249.892215568863
Music: 56482.02985074627
Food & Drink: 20179.093023255813
Book: 8498.333333333334
Lifestyle: 8978.308510638299
Entertainment: 10822.961077844311
Social Networking: 53078.195804195806
Reference: 67447.9
Health & Fitness: 19952.315789473683
Sports: 20128.974683544304
Games: 18924.68896765618
Productivity: 19053.887096774193
Utilities: 14010.100917431193


In [33]:
fully_cleaned_data_free[5]

['Smoke Effect Photo Maker - Smoke Editor',
 'ART_AND_DESIGN',
 '3.8',
 '178',
 '19M',
 '50,000+',
 'Free',
 '0',
 'Everyone',
 'Art & Design',
 'April 26, 2018',
 '1.1',
 '4.0.3 and up']

In [45]:
android_freq_table = freq_table(fully_cleaned_data_free, 1)
print(android_freq_table)

{'ENTERTAINMENT': 0.009589350180505414, 'SOCIAL': 0.026624548736462094, 'PERSONALIZATION': 0.03316787003610108, 'PARENTING': 0.006543321299638989, 'VIDEO_PLAYERS': 0.017937725631768955, 'WEATHER': 0.008009927797833934, 'NEWS_AND_MAGAZINES': 0.027978339350180504, 'FAMILY': 0.18907942238267147, 'MEDICAL': 0.03531137184115524, 'SPORTS': 0.03395758122743682, 'AUTO_AND_VEHICLES': 0.009250902527075812, 'MAPS_AND_NAVIGATION': 0.013989169675090252, 'TRAVEL_AND_LOCAL': 0.023352888086642598, 'LIFESTYLE': 0.039034296028880866, 'PHOTOGRAPHY': 0.02944494584837545, 'COMICS': 0.006204873646209386, 'LIBRARIES_AND_DEMO': 0.009363718411552346, 'FOOD_AND_DRINK': 0.012409747292418772, 'DATING': 0.01861462093862816, 'COMMUNICATION': 0.032378158844765345, 'BUSINESS': 0.04591606498194946, 'FINANCE': 0.03700361010830325, 'SHOPPING': 0.022450361010830325, 'EVENTS': 0.007107400722021661, 'HEALTH_AND_FITNESS': 0.030798736462093863, 'BOOKS_AND_REFERENCE': 0.021435018050541516, 'PRODUCTIVITY': 0.03892148014440433,

In [52]:
for category in android_freq_table:
    total_rating_points = 0
    len_category = 0
    
    for row in fully_cleaned_data_free:
        
        category_app = row[1]
        
        if category_app == category:
            installs = row[5]
            
            installs = installs.replace('+', '')
            installs = installs.replace(',', '')
            
            print(installs)
            
            total_installs += installs
            #len_category += 1

            #print(total_rating_points)                           
#    print(genre +': ' + str(total_rating_points/len_category))

1000000
1000000
10000000
5000000
5000000
10000000
5000000
100000000
5000000
1000000
5000000
5000000
1000000
1000000
10000000
100000000
5000000
10000000
1000000
50000000
5000000
1000000
10000000
1000000
10000000
50000000
1000000
100000000
50000000
1000000
5000000
50000000
100000000
100000
1000000
1000000
10000000
100000
100000
1000000
100000
100000
1000000
1000000
1000000
10000000
10000000
10000000
50000
1000000
1000000
5000000
1000000
1000000
1000000
1000000
1000000
10000000
10000000
10000000
500000
1000000
1000000
1000000
10000000
1000000
100000000
10000000
1000000
10000000
10000000
100000
1000000
10000
100000
100000
5000000
100000
1000000
1000000
1000000
1000000
10000000
10000000
10000000
1000000000
500000000
100000000
100000
100000000
10000000
1000000000
1000000
1000000
1000000
5000000
100000
1000000
5000000
1000000
5000000
10000000
5000000
1000000
500000
5000000
50000
1000000
5000000
1000000
1000000
10000000
10000000
10000000
1000000
1000000
10000000
5000000
100000000
10000000
1000

In [49]:
android_header

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']